# 一 任务描述及方法设计

## 1 Flat & Nested NER 任务的区别

嵌套命名实体识别（Nested NER）是NER中的一种特殊情况。

解决Flat NER（常规NER）任务的方法通常是序列标注，
即让模型给每一个token打上标签，
常用的打标签方法有两种（BIO和BIOES），
例如使用BIOES给“我住在广州市海珠区。”这句话中的每一个token打上标签就是：

> Example 1:    
> 我住在广州市海珠区。  
> O, O, O, B-Loc，I-Loc，E-Loc，B-Loc，I-Loc，E-Loc, O。    

其中O表示“该token不属于实体”，B表示“实体的开头token”，I表示“实体的中间token”，E表示“实体的结尾token”，S表示“只有一个token的实体”，Loc表示实体的标签类型为地名。    

那 Nested NER 任务是什么？
以我们接下来要使用的ACE2004数据集中的一个句子为例（暂时找不到用于验证Nested NER任务的中文数据集，MSRA-ch和onto04-ch都是不存在嵌套实体的）：

> Example 2:    
> The Chinese government and the Australian government signed an agreement today , wherein the Australian party would provide China with a preferential financial loan of 150 million Australian dollars . 

> [[0, 2], [1, 1], [4, 6], [5, 5], [13, 15], [14, 14], [18, 18], [27, 27]]

其中3处地方存在嵌套实体：
> 实体"The Chinese government"\[0,2\]嵌套了实体"Chinese"\[1,1\].  
> 实体"The Australian government"\[4,6\]嵌套了实体"Australian"\[5,5\].       
> 实体"The Australian party"\[13,15\]嵌套了实体"Australian"\[14,14\]. 

对于这种情况，
如果继续使用BIOES等方法让序列标注模型为句子打上标签，
那么任务就从单标签任务变成了多标签任务，
如"Chinese"同时具有"S-GPE"和"I-GPE"两种标签，
但相关工作表明，
这种处理方法比较麻烦且效果也不理想。

所以在2020年，[基于MRC（机器阅读理解）思想来解决Nested NER任务的框架](https://arxiv.org/pdf/1910.11476.pdf)被提出。

## 2 基于 MRC 框架解决 Nest NER 任务

MRC 是指给模型一段文本，然后指定一个问题，让模型在文本中找出该问题的答案。
比如在 Nested NER 中，对于`Example 2`的这句话，可以设置以下的问题：

> Input1: Find an organization such as company, agency and insititution in the context. 

模型返回的是实体所在位置（span），如下所示。

> Output2: [0, 2], [4, 6], [13, 15]  

接着对于其它实体，可以再问：

> Input2: Find an country...    
> Output2: [1, 1], [5, 5], [14, 14], [18, 18], [27, 27] 

所以 Nested NER 任务就转换成了多轮次的问答任务（A multi-turn QA task）。

## 3 模型设计

所以我们现在的目标是设计这样一个模型：**输入**一个问题和一段文本，
**返回**实体在文本中的具体位置。

基于 MRC 的数据格式与常规 NER 的数据格式不太一样，
从常规 NER 转换到 MRT 的格式需要一些特殊处理
（在下面的代码实现中将会看到 MRC 格式数据的具体样子）。

简单来说，每一个样本需要转换成如下形式的三元组表示：

$$
(\mathrm{QUESTION},\mathrm{ANSWER},\mathrm{CONTEXT})
$$

$\mathrm{QUESTION}$ 就是将原来的token标签$y \in Y$转换成对应的自然语言问题描述$q_y = \{q_1, q_2, \cdots, q_m\}$，如上述的*Input1*一样。

$\mathrm{CONTEXT}$ 是文本描述，记为 $ X = \{x_1, x_2, \cdots, x_n \} $。

$\mathrm{ANSWER}$ 是实体所在位置，记为 $x_{start,end}=\{x_{start}, x_{start+1}, \cdots, x_{end_1}, x_{end}\}$，实际上是$X$的一个子串。

因此该单个样本的三元组表示为：

$$(q_y, x_{start,end}, X)$$

其中 $q_y$ 和 $X$ 作为模型的输入，$x_{start,end}$ 作为模型的输出。

因为模型是基于 BERT 来实现，所以输入 BERT 的 token 是：

$$[CLS], q_1, q_2, \cdots, q_m, [SEP], x_1, x_2, \cdots, x_n, [SEP]$$

经过 BERT 得到词嵌入 $E \in \mathbb{R}^{n \times d}$ 之后（$d$ 是词嵌入维度），
需要将所有的词嵌入丢进 3 个 token 的二分类器中，分别计算实体的开始位置（start index）、结束位置（end index）和它们的位置匹配（span match）。

### 3.1 确定所有实体的开始和结束位置

实体的开始和结束位置各使用一个二分器来确定，它们接收token的嵌入表示，然后判断该token是否是实体的开始（结束）位置。

判断 token 是否是开始位置的二分器的形式化描述如下：
$$
P_{start} = \mathrm{softmax}_{each row} (E T_{start}) \in \mathbf{R}^{n \times 2}
$$

判断 token 是否是结束位置的二分器的形式化描述如下：
$$
P_{end} = \mathrm{softmax}_{each row} (E T_{end}) \in \mathbf{R}^{n \times 2}
$$

其中$T_{start}, T_{end} \in \mathbb{R}^{d \times 2}$是可学习的参数矩阵。
$P_{start}^i, P_{end}^j \in \mathbb{R}^{1 \times 2}$ 分别表示第 $i(j)$ 个 token 是 start(end) index 和不是 start(end) index 的概率。

### 3.2 匹配对应的开始和结束位置

因为存在嵌套实体，因此不能按照先后顺序去匹配得到的开始和结束位置。

所以使用第3个二分类器来判断所选的$E_{i_{start}}$和$E_{j_{end}}$的匹配概率：

$$P_{i_{start}, j_{end}} = \mathrm{sigmoid}(\mathbf{m} [E_{i_{start}} \Vert E_{j_{end}}])$$
其中$m \in \mathbb{R}^{1 \times 2d}$。

**注意**：

在实际的代码实现中，$T_{start}, T_{end}, \in \mathbb{R}^{d \times 1}$，

$m$ 是两层的 MLP 实现。

### 3.3 计算损失函数

最终的损失值是3个二分器各自损失值的加权累加，形式化描述如下：

$$
\mathcal{L}_{start} = \mathrm{CE}(P_{start}, Y_{start})
$$

$$
\mathcal{L}_{end} = \mathrm{CE}(P_{end}, Y_{end})
$$

$$
\mathcal{L}_{span} = \mathrm{CE}(P_{start, end}, Y_{start, end})
$$
$$
\mathcal{L} = \alpha \mathcal{L}_{start} + \beta \mathcal{L}_{end} + \gamma \mathcal{L}_{span}
$$

其中$\alpha,\beta,\gamma$是超参数，原论文均设为1。

最终损失值的计算在代码实现中会更复杂，因为要考虑负样本和正样本的数量不平衡问题。

# 二 代码实现

In [1]:
from google.colab import drive
drive.mount('/content/drive')

! pip install transformers

Mounted at /content/drive
     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 42.5 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 6.5 MB 39.1 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import json
import torch
import os
from tokenizers import BertWordPieceTokenizer
from torch.utils.data import Dataset, DataLoader
from typing import List
import numpy as np
import torch.nn as nn
from torch.nn import functional as F
from transformers import BertModel, BertPreTrainedModel, BertConfig, AdamW
from torch.nn.modules import BCEWithLogitsLoss

## 1 数据集介绍

从原论文中下载了 6 个数据集，
其中ACE2004、ACE2005和CONII03存在重叠实体，
GENIA、MSRA-ch和ONTO4-ch不存在重叠实体，
它们均已经被处理成MRC-NER的格式，
各数据集MRC格式的下载链接如下：

- [MSRA-ch](https://drive.google.com/file/d/1bAoSJfT1IBdpbQWSrZPjQPPbAsDGlN2D/view?usp=sharing)
- [ONTO4-ch](https://drive.google.com/file/d/1CRVgZJDDGuj0O1NLK5DgujQBTLKyMR-g/view?usp=sharing)
- [CONLL03](https://drive.google.com/file/d/1mGO9CYkgXsV-Et-hSZpOmS0m9G8A5mau/view?usp=sharing)
- [ACE2004](https://drive.google.com/file/d/1U-hGOgLmdqudsRdKIGles1-QrNJ7SSg6/view?usp=sharing)
- [ACE2005](https://drive.google.com/file/d/1iodaJ92dTAjUWnkMyYm8aLEi5hj3cseY/view?usp=sharing)
- [GENIA](https://drive.google.com/file/d/1oF1P8s-0MN9X1M1PlKB2c5aBtxhmoxXb/view?usp=sharing)

因为暂时找不到存在嵌套实体的中文数据集，
所以接下来以英文数据集 ACE2004 作为例子。

---

首先来看一下该数据集具体长什么样子。

In [3]:
project_path = '/content/drive/MyDrive/mrc-bert-ner/'
data_name = ['/ACE2004', '/ACE2005', '/CONII03', '/GENIA', '/MSRA-ch', '/ONTO4-ch', '/debug'][0]  # 选择所要验证的数据集
file_path = [f'{project_path}data{data_name}/mrc-ner.train',f'{project_path}data{data_name}/mrc-ner.dev', f'{project_path}data{data_name}/mrc-ner.test']
with open(file_path[0]) as f: data = json.load(f)
data[7]

{'context': 'The Chinese government and the Australian government signed an agreement today , wherein the Australian party would provide China with a preferential financial loan of 150 million Australian dollars .',
 'end_position': [2, 1, 6, 5, 15, 14, 18, 27],
 'entity_label': 'GPE',
 'impossible': False,
 'qas_id': '1.1',
 'query': 'geographical political entities are geographical regions defined by political and or social groups such as countries, nations, regions, cities, states, government and its people.',
 'span_position': ['0;2',
  '1;1',
  '4;6',
  '5;5',
  '13;15',
  '14;14',
  '18;18',
  '27;27'],
 'start_position': [0, 1, 4, 5, 13, 14, 18, 27]}

这是在Example2中的那句话，可以看到一个样本用字典来保存信息，其中各key的含义如下：
* context：句子的自然语言内容。
* end_position：各实体结束的位置。
* entity_label：实体的标签，这里的GPE指地理政治实体
* impossible：False代表该句子存在实体，True代表该句子不存在实体。
* qas_id：由于一条句子可以根据具体问题划分成多个样本，所以上面的1.1表示句子1中的第1个样本。
* query：指输入到模型的自然语言问题。
* span_position：具体实体所在的边界范围，比如'0;2'表示"The Chinese government"是一个GPE。
* start_position：各实体开始的位置。

In [4]:
"""数据集中有哪些entity label，以及对应的query是什么，以及属于它们的样本的数量情况。"""
queries = {}
queries_count = {}
for d in data:
    if d['entity_label'] not in queries:
        queries[d['entity_label']] = d['query']
        queries_count[d['entity_label']] = 1
    if d['entity_label'] in queries_count:
        queries_count[d['entity_label']] += 1
print('entity label, count, query:')
for k in queries:
    print(k, queries_count[k], queries[k])

entity label, count, query:
GPE 6203 geographical political entities are geographical regions defined by political and or social groups such as countries, nations, regions, cities, states, government and its people.
ORG 6203 organization entities are limited to companies, corporations, agencies, institutions and other groups of people.
PER 6203 a person entity is limited to human including a single individual or a group.
FAC 6203 facility entities are limited to buildings and other permanent man-made structures such as buildings, airports, highways, bridges.
VEH 6203 vehicle entities are physical devices primarily designed to move, carry, pull or push the transported object such as helicopters, trains, ship and motorcycles.
LOC 6203 location entities are limited to geographical entities such as geographical areas and landmasses, mountains, bodies of water, and geological formations.
WEA 6203 weapon entities are limited to physical devices such as instruments for physically harming such

可以看到ACE2004的训练集中一共有7种类型的NER（GPE、ORG、PER、FAC、VEH、LOC和WEA）。
且各类样本的总数均是6203个。

---

接下来统计整个数据集中存在实体的句子有多少条，以及存在重叠实体的句子又有多少条。

In [5]:
"""验证数据集中的重叠实体情况"""
# """step1：把从训练集、验证集和测试集中读出的数据都累加到一起。"""
with open(file_path[0]) as f: data = json.load(f)
with open(file_path[1]) as f: data += json.load(f)
with open(file_path[2]) as f: data += json.load(f)

# """step2：遍历整个数据集，建立以context作为key，span_position作为value的字典，并且只有存在实体的句子才会被加入字典之中。"""
context2spans = {}
for d in data:
    tmp_context = d['context']
    tmp_span = d['span_position']
    if not tmp_span:  # 如果没有实体就跳过 
        continue  
    else:  # 如果有实体，就将span的字符串形式转换为列表形式，例如将["1;3", "2;4"]转换为[[1,3], [2,4]]。目的是为了方便排序之后检测重叠实体。
        tmp_span = [[int(s.split(';')[0]), int(s.split(';')[1])] for s in tmp_span]  

    # 因为在MRC的格式中，一条相同的句子根据Query的不同可能被分成多个样本，所以需要把指向同一条句子的样本都放到同一个key中。
    if tmp_context in context2spans:  
        context2spans[tmp_context].extend(tmp_span)
    else: 
        context2spans[tmp_context] = tmp_span

# """step3：检测重叠实体，方法是将列表形式化后的span排序，然后检查第i个span的start是否小于等于第i-1个span的end，如果是，则把相应的key和value存入nested_example字典中。"""
nested_example = {}
for k in context2spans:
    span = context2spans[k]
    span.sort()

    for i in range(1, len(span)):
        if span[i-1][1] >= span[i][0]:
            nested_example[k] = span
            break

print(f'数据集：{data_name[1:]}中有实体的句子有{len(context2spans)}条，但有嵌套实体的句子有{len(nested_example)}条，例如下面就是一些嵌套句子：')

breakpoint = 5
for k in nested_example:
    print(k, '\n' ,nested_example[k])
    breakpoint -= 1
    if breakpoint == 0:
        break

数据集：ACE2004中有实体的句子有6933条，但有嵌套实体的句子有3408条，例如下面就是一些嵌套句子：
The Chinese government and the Australian government signed an agreement today , wherein the Australian party would provide China with a preferential financial loan of 150 million Australian dollars . 
 [[0, 2], [1, 1], [4, 6], [5, 5], [13, 15], [14, 14], [18, 18], [27, 27]]
Lasting for two days , the ' 94 Development Assistance Cooperation Annual Meeting between the Chinese government and the Australian government concluded today in Melbourne . 
 [[14, 16], [15, 15], [18, 20], [19, 19], [24, 24]]
The Chinese delegation with Yongtu Long , assistant minister of the Ministry of Foreign Economy and Trade , as the delegation leader , and the Australian delegation with Flad , director general of Australian International Development Bureau Assistance Department as the delegation leader , chaired the meeting . 
 [[0, 21], [1, 1], [4, 5], [7, 16], [10, 16], [19, 21], [20, 20], [24, 42], [25, 25], [28, 28], [30, 38], [33, 38], [40, 42], [41

## 2 数据预处理

在模型训练开始之前，
需要将样本中的自然语言转换成模型可以计算的编码表示。

`torch`提供了`DataLoader`来处理数据集，
但NLP的样本处理比较麻烦，
所以在将数据丢进`DataLoader`之前，
还需要对数据集做一些预处理，
具体的预处理过程如下面的`MRCNERDataset(Dataset)`类所示。

`MRCNERDataset(Dataset)`类的主要作用是：
* 将原数据转换成 BERT 需要的输入格式，
* 将实体开始位置、结束位置和span match转换成向量表示。

`MRCNERDataset(Dataset)`类的具体解释见代码注释。

In [6]:
class MRCNERDataset(Dataset):
    """
    Args:
        json_path: mrc-ner格式的json文件路径。
        tokenizer: Bert的分词器。
        max_length: 输入Bert的最大序列长度，指word-piece之后的query + context + 3的最大长度
        possible_only: 如果设置为True，则只使用存在答案的样本（即存在实体的样本）。
        is_chinese: 是否是中文数据集，中文数据集需要去掉在token之间的空格。
        pad_to_maxlen: 是否填充至最大长度。
    Returns:
        tokens: tokens of query + context, shape is [seq_len].
        token_type_ids: token type ids, 0 for query, 1 for context, shape is [seq_len].
        start_labels: start labels of NER in tokens, shape is [seq_len].
        end_labels: end labels of NER in tokens, shape is [seq_len].
        label_mask: label mask, 1 for counting into loss, 0 for ignoring. shape is [seq_len].
        match_labels: match labels, shape is [seq_len, seq_len]. match_labels[i][j]==1意味着从i到j存在实体。
        sample_idx: sample id
        label_idx: label id
    """
    def __init__(self, json_path, tokenizer: BertWordPieceTokenizer, max_length: int = 512, possible_only=False, 
                 is_chinese=False, pad_to_maxlen=False):
        self.all_data = json.load(open(json_path, encoding='utf-8'))
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.possible_only = possible_only
        if self.possible_only:
            self.all_data = [x for x in self.all_data if x['start_position']]
        self.is_chinese = is_chinese
        self.pad_to_maxlen = pad_to_maxlen

    def __len__(self):
        return len(self.all_data)
    
    def __getitem__(self, item):
        
        tokenizer = self.tokenizer

        # Step 1: 读取样本存储在字典中的相关值
        data = self.all_data[item]
        qas_id = data.get('qas_id', '0.0') 
        sample_idx, label_idx = qas_id.split('.')
        sample_idx = torch.LongTensor([int(sample_idx)])
        label_idx = torch.LongTensor([int(label_idx)])
        query = data['query']
        context = data['context']
        start_positions = data['start_position']
        end_positions = data['end_position']

        # Step 2: 区分处理中文和英文数据集，原因如下：
        #   中文数据集中context的字符是用空格隔开的(query不存在这个问题)，如"上 海 浦 东 开 发 与 法 制 建 设 同 步"。
        #   并且中文所用到预训练模型会将空格也当做一个token处理，所以需要去掉中文context中的空格。
        #   另外，英文数据集的分词考虑了空格，而原来的开始位置和结束位置没有考虑空格，所以它们需要重新映射。
        if self.is_chinese:
            context == ''.join(context.split())
            end_positions = [x+1 for x in end_positions]
        else:
            words = context.split()
            start_positions = [x + sum([len(w) for w in words[:x]]) for x in start_positions]
            end_positions = [x + sum([len(w) for w in words[:x + 1]]) for x in end_positions]

        # Step 3: 用 BERT 分词器处理 query + context。
        query_context_tokens = tokenizer.encode(query, context, add_special_tokens=True)
        tokens = query_context_tokens.ids
        type_ids = query_context_tokens.type_ids
        offsets = query_context_tokens.offsets
        
        # Step 4: 重新修正开始位置和结束位置，原因如下：
        #   - 因为添加了query在前面。
        #   - 英文的分词是根据词根词缀划分的，即word-piece tokenize。
        origin_offset2token_idx_start = {}
        origin_offset2token_idx_end = {}
        for token_idx in range(len(tokens)):
            if type_ids[token_idx] == 0:  # 跳过query的token
                continue
            token_start, token_end = offsets[token_idx]
            if token_start == token_end == 0:  # 跳过[CLS]和[SEP]等特殊token
                continue
            origin_offset2token_idx_start[token_start] = token_idx
            origin_offset2token_idx_end[token_end] = token_idx
        new_start_positions = [origin_offset2token_idx_start[start] for start in start_positions]
        new_end_positions = [origin_offset2token_idx_end[end] for end in end_positions]

        # Step 5: 建立开始位置、结束位置的mask以及向量表示。
        label_mask = [
            (0 if type_ids[token_idx] == 0 or offsets[token_idx] == (0, 0) else 1)
            for token_idx in range(len(tokens))
        ]
        start_label_mask = label_mask.copy()
        end_label_mask = label_mask.copy()
        # 同样是由于work-piece的问题，英文的mask需要特殊处理（中文不需要，中文的start_label_mask和end_label_mask是一样的）。
        # 比如对于'xinhua'这个单词，它被work-piece分成'xi ##nh ##ua'三个token，
        # 其对应的 start_label_mask 和 end_label_mask 分别是 [1,0,0] 和 [0,0,1]。
        if not self.is_chinese:  
            for token_idx in range(len(tokens)):
                current_word_idx = query_context_tokens.word_ids[token_idx]
                next_word_idx = query_context_tokens.word_ids[token_idx+1] if token_idx+1 < len(tokens) else None
                prev_word_idx = query_context_tokens.word_ids[token_idx-1] if token_idx-1 > 0 else None
                if prev_word_idx is not None and current_word_idx == prev_word_idx:
                    start_label_mask[token_idx] = 0
                if next_word_idx is not None and current_word_idx == next_word_idx:
                    end_label_mask[token_idx] = 0
        assert all(start_label_mask[p] != 0 for p in new_start_positions)
        assert all(end_label_mask[p] != 0 for p in new_end_positions)
        assert len(new_start_positions) == len(new_end_positions) == len(start_positions)
        assert len(label_mask) == len(tokens)
        start_labels = [(1 if idx in new_start_positions else 0) for idx in range(len(tokens))]  # 开始位置的向量表示
        end_labels = [(1 if idx in new_end_positions else 0) for idx in range(len(tokens))]  # 结束位置的向量表示

        # Step 6: 按照句子最大长度截断（如果超出）、并保证最后一个token是[SEP]。
        tokens = tokens[: self.max_length]
        type_ids = type_ids[: self.max_length]
        start_labels = start_labels[: self.max_length]
        end_labels = end_labels[: self.max_length]
        start_label_mask = start_label_mask[: self.max_length]
        end_label_mask = end_label_mask[: self.max_length]
        sep_token = tokenizer.token_to_id('[SEP]')
        if tokens[-1] != sep_token:
            assert len(tokens) == self.max_length
            tokens = tokens[: -1] + [sep_token]
            start_labels[-1] = 0
            end_labels[-1] = 0
            start_label_mask[-1] = 0
            end_label_mask[-1] = 0

        # Step 7: 填充句子
        if self.pad_to_maxlen:
            tokens = self.pad(tokens, 0)
            type_ids = self.pad(type_ids, 1)
            start_labels = self.pad(start_labels)
            end_labels = self.pad(end_labels)
            start_label_mask = self.pad(start_label_mask)
            end_label_mask = self.pad(end_label_mask)
        
        # Step 8: 生成 span match 矩阵
        seq_len = len(tokens)
        match_labels = torch.zeros([seq_len, seq_len], dtype=torch.long)
        for start, end in zip(new_start_positions, new_end_positions):
            if start >= seq_len or end >= seq_len:
                continue
            match_labels[start, end] = 1
        
        return [
            torch.LongTensor(tokens),
            torch.LongTensor(type_ids),
            torch.LongTensor(start_labels),
            torch.LongTensor(end_labels),
            torch.LongTensor(start_label_mask),
            torch.LongTensor(end_label_mask),
            match_labels,
            sample_idx,
            label_idx
        ]
        
    def pad(self, lst, value=0, max_length=None):
        max_length = max_length or self.max_length
        while len(lst) < max_length:
            lst.append(value)
        return lst

def collate_to_max_length(batch: List[List[torch.Tensor]]) -> List[torch.Tensor]:
    """
    若当前batch中样本的最大句子长度为batch_max_seq_length < max_seq_length，
    则当前batch的其余句子均填充到batch_max_seq_length即可。

    pad to maximum length of this batch
    Args:
        batch: a batch of samples, each contains a list of field data(Tensor):
            tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx
    Returns:
        output: list of field batched data, which shape is [batch, max_length]
    """
    batch_size = len(batch)
    max_length = max(x[0].shape[0] for x in batch)
    output = []

    for field_idx in range(6):
        pad_output = torch.full([batch_size, max_length], 0, dtype=batch[0][field_idx].dtype)
        for sample_idx in range(batch_size):
            data = batch[sample_idx][field_idx]
            pad_output[sample_idx][: data.shape[0]] = data
        output.append(pad_output)

    pad_match_labels = torch.zeros([batch_size, max_length, max_length], dtype=torch.long)
    for sample_idx in range(batch_size):
        data = batch[sample_idx][6]
        pad_match_labels[sample_idx, : data.shape[1], : data.shape[1]] = data
    output.append(pad_match_labels)

    output.append(torch.stack([x[-2] for x in batch]))
    output.append(torch.stack([x[-1] for x in batch]))

    return output


下面函数显示了`MRCNERDataset(Dataset)`类对具体数据的处理结果。

In [7]:
def show_data():
    """查看MERNERDataset类对原数据处理后的结果"""

    # 中文数据集
    bert_path = project_path + "bert-base-chinese"
    vocab_file = os.path.join(bert_path, "vocab.txt")
    json_path = file_path[0]  # 训练集、验证集、测试集分别对应下标0，1，2
    is_chinese = True

    # 若该数据集是英文数据集，请把下面4语句注释掉。
    bert_path = project_path + "bert-base-english"
    json_path = file_path[0]
    vocab_file = os.path.join(bert_path, "vocab.txt")
    is_chinese = False

    tokenizer = BertWordPieceTokenizer(vocab_file)
    dataset = MRCNERDataset(json_path=json_path, tokenizer=tokenizer, is_chinese=is_chinese)

    dataloader = DataLoader(dataset, batch_size=1, collate_fn=collate_to_max_length)
    
    for batch in dataloader:
        for tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx in zip(*batch):  # 让数据逐条析出
            
            if sample_idx != 1 or label_idx != 1:  # 是为了只显示Example2所在的那个样本。
                continue

            print('-*-' * 5, 'MRCNERDataset对Example2中那条句子处理后的结果如下', '-*-' * 5)
            print('sample_idx:',sample_idx.item())
            print('label_idx:', label_idx.item())
            print('word-piece之后的长度:', len(tokens))

            tokens = tokens.tolist()
            print('=='*3, 'query+contenxt编码前的表示:','=='*3, '\n', tokenizer.decode(tokens, skip_special_tokens=False))
            print('=='*3, 'query+contenxt编码后的表示:','=='*3, '\n', tokens)
            print('=='*3,'token_type_ids:','=='*3, '\n', token_type_ids.tolist())
            print('=='*3,'start_labels:','=='*3, '\n', start_labels.tolist())
            print('=='*3,'end_labels:','=='*3,'\n', end_labels.tolist())
            print('=='*3,'start_label_mask:','=='*3, '\n', start_label_mask.tolist())
            print('=='*3,'end_label_mask:','=='*3,'\n', end_label_mask.tolist())
            print('=='*3, 'match_labels中等于1的位置', '=='*3)
            print(np.argwhere(match_labels.numpy() == 1).tolist())
            return

show_data()

-*--*--*--*--*- MRCNERDataset对Example2中那条句子处理后的结果如下 -*--*--*--*--*-
sample_idx: 1
label_idx: 1
word-piece之后的长度: 64
====== query+contenxt编码前的表示: ====== 
 [CLS] geographical political entities are geographical regions defined by political and or social groups such as countries, nations, regions, cities, states, government and its people. [SEP] the chinese government and the australian government signed an agreement today, wherein the australian party would provide china with a preferential financial loan of 150 million australian dollars. [SEP]
====== query+contenxt编码后的表示: ====== 
 [101, 10056, 2576, 11422, 2024, 10056, 4655, 4225, 2011, 2576, 1998, 2030, 2591, 2967, 2107, 2004, 3032, 1010, 3741, 1010, 4655, 1010, 3655, 1010, 2163, 1010, 2231, 1998, 2049, 2111, 1012, 102, 1996, 2822, 2231, 1998, 1996, 2827, 2231, 2772, 2019, 3820, 2651, 1010, 16726, 1996, 2827, 2283, 2052, 3073, 2859, 2007, 1037, 9544, 24271, 3361, 5414, 1997, 5018, 2454, 2827, 6363, 1012, 102]
====== token_type_ids: ===

## 3 模型实现

如 一.3 模型设计中所介绍，模型主要包括BERT和三个二分类器。

在下面的代码实现中，模型具体包括的部分如下：
- 作为词嵌入的 BERT。（对应`BertQueryNER(BertPreTrainedModel)`类中的`self.bert`）
- 二分类 $token_i$ 是否是开始位置的 Start Outpus Classifier。（对应`BertQueryNER(BertPreTrainedModel)`类中的`self.start_outputs`）
- 二分类 $token_j$ 是否是结束位置的 End Outpus Classifier。（对应`BertQueryNER(BertPreTrainedModel)`类中的`self.end_outputs`）
- 二分类 $tokens_{i,j}$(start token和end token的词嵌入拼接表示)是否匹配的 Span Match Output Classifier。（对应`BertQueryNER(BertPreTrainedModel)`类中的`self.span_embedding`，具体的实现在`MultiNonLinearClassifier`类）
- 3 个Classifier输出的损失计算（对于函数`compute_loss`）。

**细节见代码注释。**

In [8]:
class BertQueryNER(BertPreTrainedModel):
    def __init__(self, config):
        super(BertQueryNER, self).__init__(config)

        self.bert = BertModel(config)  # Bert
        self.start_outputs = nn.Linear(config.hidden_size, 1)  # 开始位置分类器 Start Output Classifier
        self.end_outputs = nn.Linear(config.hidden_size, 1)  # 结束位置分类器 End Output Classifier
        self.span_embedding = MultiNonLinearClassifier(config.hidden_size * 2, 1,  # 边界匹配分类器 Span Match Output Classifier
                                                       config.mrc_dropout, 
                                                       intermediate_hidden_size=config.classifier_intermediate_hidden_size)

        self.hidden_size = config.hidden_size

        self.init_weights()  # 权重初始化

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        """
        Args:
            input_ids: bert input tokens, tensor of shape [seq_len]
            token_type_ids: 0 for query, 1 for context, tensor of shape [seq_len]
            attention_mask: attention mask, tensor of shape [seq_len]
        Returns:
            start_logits: start/non-start probs of shape [seq_len]
            end_logits: end/non-end probs of shape [seq_len]
            match_logits: start-end-match probs of shape [seq_len, 1]
        """

        # Step 1: 获得所有token的BERT词嵌入
        bert_outputs = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        sequence_heatmap = bert_outputs[0]  # [batch, seq_len, hidden]
        batch_size, seq_len, hid_size = sequence_heatmap.size()

        # Step 2: 计算 tokens 是开始位置的 logits 预测值
        start_logits = self.start_outputs(sequence_heatmap).squeeze(-1)  # [batch, seq_len, 1]

        # Step 3: 计算 tokens 是结束位置的 logits 预测值
        end_logits = self.end_outputs(sequence_heatmap).squeeze(-1)  # [batch, seq_len, 1]

        # Step 4: 逐个拼接句子中所有的 tokens 为 Span Match Output Classifier 做准备，
        #         最后做成一个shape为[batch, seq_len, seq_len, hidden*2]的张量span_matrix。
        start_extend = sequence_heatmap.unsqueeze(2).expand(-1, -1, seq_len, -1)  # [batch, seq_len, seq_len, hidden]
        end_extend = sequence_heatmap.unsqueeze(1).expand(-1, seq_len, -1, -1)  # [batch, seq_len, seq_len, hidden]
        span_matrix = torch.cat([start_extend, end_extend], 3)  # [batch, seq_len, seq_len, hidden*2]
        
        # Step 5: 计算 span matrix 中的 logits 预测值，
        #         从 span matrix 中的 [batch, seq_len, seq_len, hidden*2] 变成 span_logits 中的[batch, seq_len, seq_len]
        #         其中 span_logits[i][j] 表示第 i 个 tokens 作为开始位置，第 j 个 tokens 作为结束位置的匹配预测值。
        span_logits = self.span_embedding(span_matrix).squeeze(-1)  # [batch, seq_len, seq_len]
        

        return start_logits, end_logits, span_logits

class MultiNonLinearClassifier(nn.Module):
    def __init__(self, hidden_size, num_label, dropout_rate, act_func="gelu", intermediate_hidden_size=None):
        super(MultiNonLinearClassifier, self).__init__()
        
        self.num_label = num_label
        self.intermediate_hidden_size = hidden_size if intermediate_hidden_size is None else intermediate_hidden_size
        self.classifier1 = nn.Linear(hidden_size, self.intermediate_hidden_size)
        self.classifier2 = nn.Linear(self.intermediate_hidden_size, self.num_label)
        self.dropout = nn.Dropout(dropout_rate)
        self.act_func = act_func

    def forward(self, input_features):
        """
        令 input_features 为 X，output_features 为 O，则forward的逻辑就是一个MLP：
        O = W2 \cdot dropout(activete(W1 \cdot X))
        """
        features_output1 = self.classifier1(input_features)

        if self.act_func == "gelu":
            features_output1 = F.gelu(features_output1)
        elif self.act_func == "relu":
            features_output1 = F.relu(features_output1)
        elif self.act_func == "tanh":
            features_output1 = F.tanh(features_output1)
        else:
            raise ValueError
        features_output1 = self.dropout(features_output1)
        features_output2 = self.classifier2(features_output1)
        return features_output2

class BertQueryNerConfig(BertConfig):
    def __init__(self, **kwargs):
        super(BertQueryNerConfig, self).__init__(**kwargs)
        self.mrc_dropout = kwargs.get("mrc_dropout", 0.1)
        self.classifier_intermediate_hidden_size = kwargs.get("classifier_intermediate_hidden_size", 1024)
        self.classifier_act_func = kwargs.get("classifier_act_func", "gelu")

def compute_loss(start_logits, end_logits, span_logits, 
                 start_labels, end_labels, match_labels, 
                 start_label_mask, end_label_mask):
    """
    分别计算start_logits、end_logits和span_logits的损失值，这里是计算所有位置上的损失，
    但考虑到负样本是多数类，正样本是少数类，所以实际上还有更好的处理方法。
    """

    batch_size, seq_len = start_logits.size()
    bce_loss = BCEWithLogitsLoss(reduction='none')

    start_float_label_mask = start_label_mask.view(-1).float()  # shape=batch x n
    end_float_label_mask = end_label_mask.view(-1).float()
    match_label_row_mask = start_label_mask.bool().unsqueeze(-1).expand(-1, -1, seq_len)
    match_label_col_mask = end_label_mask.bool().unsqueeze(-2).expand(-1, seq_len, -1)
    match_label_mask = match_label_row_mask & match_label_col_mask
    match_label_mask = torch.triu(match_label_mask, 0)  # start should be less equal to end

    float_match_label_mask = match_label_mask.view(batch_size, -1).float()

    start_loss = bce_loss(start_logits.view(-1), start_labels.view(-1).float())
    start_loss = (start_loss * start_float_label_mask).sum() / start_float_label_mask.sum()
    end_loss = bce_loss(end_logits.view(-1), end_labels.view(-1).float())
    end_loss = (end_loss * end_float_label_mask).sum() / end_float_label_mask.sum()
    match_loss = bce_loss(span_logits.view(batch_size, -1), match_labels.view(batch_size, -1).float())
    match_loss = match_loss * float_match_label_mask
    match_loss = match_loss.sum() / (float_match_label_mask.sum() + 1e-10)

    return start_loss, end_loss, match_loss

## 4 模型训练

英文所用预训练模型的下载链接是：
https://huggingface.co/bert-base-uncased

中文所用预训练模型的下载链接是：
https://huggingface.co/bert-base-chinese

In [11]:
def query_span_f1(start_preds, end_preds, match_logits, start_label_mask, end_label_mask, match_labels, flat=False):
    """
    根据模型的输出，计算span的F1值。
    Args:
        start_preds: [bsz, seq_len]
        end_preds: [bsz, seq_len]
        match_logits: [bsz, seq_len, seq_len]
        start_label_mask: [bsz, seq_len]
        end_label_mask: [bsz, seq_len]
        match_labels: [bsz, seq_len, seq_len]
        flat: if True, decode as flat-ner
    Returns:
        span-f1 counts, tensor of shape [3]: tp, fp, fn
    """
    # 将0或1值转换成布尔值
    start_label_mask = start_label_mask.bool()
    end_label_mask = end_label_mask.bool()
    match_labels = match_labels.bool()
    
    bsz, seq_len = start_label_mask.size()
    
    match_preds = match_logits > 0 # [bsz, seq_len, seq_len]
    start_preds = start_preds.bool() # [bsz, seq_len]
    end_preds = end_preds.bool() # [bsz, seq_len]

    match_preds = (match_preds & start_preds.unsqueeze(-1).expand(-1, -1, seq_len) & end_preds.unsqueeze(1).expand(-1, seq_len, -1)) # 让start、end（expand之后）和match对应位置进行与运算
    match_label_mask = (start_label_mask.unsqueeze(-1).expand(-1, -1, seq_len) & end_label_mask.unsqueeze(1).expand(-1, seq_len, -1))  # 根据start和end的mask算出match的mask
    match_label_mask = torch.triu(match_label_mask, 0)  # 保证实体开始的位置小于等于结束的位置
    match_preds = match_label_mask & match_preds

    tp = (match_labels & match_preds).long().sum()  # TRUE POSITIVE
    fp = (~match_labels & match_preds).long().sum()  # FALSE POSITIVE
    fn = (match_labels & ~match_preds).long().sum()  # FALSE NEGETIVE
    return torch.stack([tp, fp, fn])

In [ ]:
# 超参数设置
alpha, beta, gamma = 1, 1, 1
EPOCHS = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 2e-5
adam_eps = 1e-8
wd = 0.01
best_dev_f1 = -1
bs = 8

# model
bert_config_dir = project_path + 'bert-base-english'
bert_config = BertQueryNerConfig.from_pretrained(bert_config_dir,
                                                 hidden_dropout_prob=0.1,
                                                 attention_probs_dropout_prob=0.1,
                                                 mrc_dropout=0.1,
                                                 classifier_act_func = 'gelu',
                                                 classifier_intermediate_hidden_size=1024)
model = BertQueryNER.from_pretrained(bert_config_dir, config=bert_config)
model.to(device)

# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": wd,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, betas=(0.9, 0.98), lr=lr, eps=adam_eps,)

# dataloader
json_path = file_path[0]
vocab_file = os.path.join(bert_config_dir, "vocab.txt")
is_chinese = False
tokenizer = BertWordPieceTokenizer(vocab_file)
train_dataset = MRCNERDataset(json_path=file_path[0], tokenizer=tokenizer, is_chinese=is_chinese)
train_dataloader = DataLoader(train_dataset, batch_size=bs, collate_fn=collate_to_max_length, shuffle=True)
dev_dataset = MRCNERDataset(json_path=file_path[1], tokenizer=tokenizer, is_chinese=is_chinese)
dev_dataloader = DataLoader(dev_dataset, batch_size=bs, collate_fn=collate_to_max_length, shuffle=False)

# 训练模型
for epoch in range(EPOCHS):

    model.train()

    for i, batch in enumerate(train_dataloader):
        batch = tuple(b.to(device) for b in batch)
        tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, _, _ = batch
        attention_mask = (tokens != 0).long()  # 只要token不是[PAD]就置1
        start_logits, end_logits, span_logits = model(tokens, token_type_ids, attention_mask)
        start_loss, end_loss, match_loss = compute_loss(start_logits=start_logits,
                                                        end_logits=end_logits,
                                                        span_logits=span_logits,
                                                        start_labels=start_labels,
                                                        end_labels=end_labels,
                                                        match_labels=match_labels,
                                                        start_label_mask=start_label_mask,
                                                        end_label_mask=end_label_mask)
        total_loss = alpha * start_loss + beta * end_loss + gamma * match_loss
        model.zero_grad()
        total_loss.backward()
        optimizer.step()

        # 定点评估验证集并保存模型
        if i > 3 and i % 2000 == 0:
            model.eval()
            dev_total_loss = 0
            dev_total_span_f1 = 0
            count_batch = 0
            
            for j, dev_batch in enumerate(dev_dataloader):
                dev_batch = tuple(dev_b.to(device) for dev_b in dev_batch)
                dev_tokens, dev_token_type_ids, dev_start_labels, dev_end_labels, dev_start_label_mask, dev_end_label_mask, dev_match_labels, _, _ = dev_batch
                dev_attention_mask = (dev_tokens != 0).long()  # 只要token不是[PAD]就置1
                with torch.no_grad():
                    
                    # 计算验证集当前batch的loss
                    dev_start_logits, dev_end_logits, dev_span_logits = model(dev_tokens, dev_token_type_ids, dev_attention_mask)
                    dev_start_loss, dev_end_loss, dev_match_loss = compute_loss(start_logits=dev_start_logits,
                                                                                end_logits=dev_end_logits,
                                                                                span_logits=dev_span_logits,
                                                                                start_labels=dev_start_labels,
                                                                                end_labels=dev_end_labels,
                                                                                match_labels=dev_match_labels,
                                                                                start_label_mask=dev_start_label_mask,
                                                                                end_label_mask=dev_end_label_mask)
                    dev_total_loss += alpha * dev_start_loss + beta * dev_end_loss + gamma * dev_match_loss

                # 计算验证集当前batch的F1值 
                span_f1_state = query_span_f1(dev_start_logits, dev_end_logits, dev_span_logits, dev_start_label_mask, dev_end_label_mask, dev_match_labels)
                all_counts = torch.stack([x for x in span_f1_state]).view(-1, 3).sum(0)
                span_tp, span_fp, span_fn = all_counts
                span_recall = span_tp / (span_tp + span_fn + 1e-10)
                span_precision = span_tp / (span_tp + span_fp + 1e-10)
                dev_total_span_f1 += span_precision * span_recall * 2 / (span_recall + span_precision + 1e-10)

                count_batch += 1

            dev_loss = dev_total_loss / count_batch  # 验证集最终的loss
            dev_span_f1 = dev_total_span_f1 / count_batch  # 验证集最终的span f1 值
            print(f'epoch:{epoch}, batch:{i}, train loss:{total_loss.item()}, dev loss:{dev_loss.item()}, dev span f1:{dev_span_f1.item()}')
            
            # 如果此次验证集的F1大于当前最好的F1，则保存模型
            if best_dev_f1 < dev_span_f1:
                best_dev_f1 = dev_span_f1
                torch.save(model, project_path + 'model_save_new.pth')
                print('SAVE!')

Some weights of the model checkpoint at /content/drive/MyDrive/mrc-bert-ner/bert-base-english were not used when initializing BertQueryNER: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertQueryNER from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertQueryNER from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertQueryNER were not initialized from the model checkpoint at /content/drive/MyDrive/mrc-bert

epoch:0, batch:2000, train loss:0.030432697385549545, dev loss:0.0378371886909008, dev span f1:0.503181517124176
SAVE!
epoch:0, batch:4000, train loss:0.029311735183000565, dev loss:0.03624585270881653, dev span f1:0.6129595041275024
SAVE!
epoch:1, batch:2000, train loss:0.004096935503184795, dev loss:0.03656528517603874, dev span f1:0.7015897631645203
SAVE!
epoch:1, batch:4000, train loss:0.008193296380341053, dev loss:0.032151203602552414, dev span f1:0.6943182945251465
epoch:2, batch:2000, train loss:0.00279993936419487, dev loss:0.04054645821452141, dev span f1:0.716867983341217
SAVE!
epoch:2, batch:4000, train loss:0.042344387620687485, dev loss:0.03349826857447624, dev span f1:0.7225610613822937
SAVE!
epoch:3, batch:2000, train loss:0.0008626565686427057, dev loss:0.04692533612251282, dev span f1:0.73361736536026
SAVE!
epoch:3, batch:4000, train loss:0.001133581972680986, dev loss:0.03900938108563423, dev span f1:0.7114824652671814


## 5 推理演示

In [24]:
def extract_nested_spans(start_preds, end_preds, match_preds, start_label_mask, end_label_mask):
    """根据模型给出的预测，返回实体所在的位置"""
    start_label_mask = start_label_mask.bool()
    end_label_mask = end_label_mask.bool()
    bsz, seq_len = start_label_mask.size()
    start_preds = start_preds.bool()
    end_preds = end_preds.bool()

    match_preds = (match_preds & start_preds.unsqueeze(-1).expand(-1, -1, seq_len) & end_preds.unsqueeze(1).expand(-1, seq_len, -1))
    match_label_mask = (start_label_mask.unsqueeze(-1).expand(-1, -1, seq_len) & end_label_mask.unsqueeze(1).expand(-1, seq_len, -1))
    match_label_mask = torch.triu(match_label_mask, 0)  # start should be less or equal to end
    match_preds = match_label_mask & match_preds
    match_pos_pairs = np.transpose(np.nonzero(match_preds.numpy())).tolist()

    return [(pos[1], pos[2]) for pos in match_pos_pairs]

In [61]:
trained_model = torch.load(project_path + 'trained_model.pth', map_location=torch.device('cpu'))

bert_config_dir = project_path + 'bert-base-english'
json_path = file_path[2]
vocab_file = os.path.join(bert_config_dir, "vocab.txt")
is_chinese = False
tokenizer = BertWordPieceTokenizer(vocab_file)

test_dataset = MRCNERDataset(json_path=json_path, tokenizer=tokenizer, is_chinese=False, possible_only=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, collate_fn=collate_to_max_length, shuffle=False)

check_ignore = 100
check_dur = 10
print('\n注意：以下位置是指query+context之后并经过word-piece处理之后的绝对位置\n')
for i, batch in enumerate(test_dataloader):
    if i > check_ignore:
        tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx = batch
        attention_mask = (tokens != 0).long()
        start_logits, end_logits, span_logits = trained_model(tokens, attention_mask=attention_mask, token_type_ids=token_type_ids)
        start_preds, end_preds, span_preds = start_logits > 0, end_logits > 0, span_logits > 0
        match_preds = span_logits > 0
        sentence = tokenizer.decode(tokens[0].tolist(), skip_special_tokens=False).split('[SEP]')
        print(f'example{i-check_ignore}:')
        print('\tQUERY  是  :', sentence[0][6:])
        print('\tCONTEXT是  :', sentence[1][1:])
        
        infer_pos = extract_nested_spans(start_preds, end_preds, match_preds, start_label_mask, end_label_mask)
        real_pos = [(pos[0], pos[1]) for pos in np.argwhere(match_labels[0].numpy() == 1)]
        
        print('\t推测的NER位置:', infer_pos)
        print('\t真实的NER位置:', real_pos)
        
        print('------' * 20)
    if i == check_ignore + check_dur:
        break



注意：以下位置是指query+context之后并经过word-piece处理之后的绝对位置

example1:
	QUERY  是  : organization entities are limited to companies, corporations, agencies, institutions and other groups of people. 
	CONTEXT是  : indonesia's reformist - minded president abdurrahman wahid has blamed the army and police for the deaths. 
	推测的NER位置: [(36, 39)]
	真实的NER位置: [(36, 39)]
------------------------------------------------------------------------------------------------------------------------
example2:
	QUERY  是  : a person entity is limited to human including a single individual or a group. 
	CONTEXT是  : indonesia's reformist - minded president abdurrahman wahid has blamed the army and police for the deaths. 
	推测的NER位置: [(17, 24), (25, 30)]
	真实的NER位置: [(17, 24), (17, 30)]
------------------------------------------------------------------------------------------------------------------------
example3:
	QUERY  是  : geographical political entities are geographical regions defined by political and or social groups 